<a href="https://colab.research.google.com/github/beltranJ/StandfordCS231n/blob/main/Image_classifier_Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Overview
Quick image classifier with CIFAR-10 Dataset using a custom CNN architecture. As a result I got a **73% accuracy at test time**.

In [18]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
import os 


Mounted at /content/drive


In [10]:
cifar_root = r"drive/MyDrive/ml/standford cs231n/assignment2/cs231n/datasets"


#Data loading and normalization

In [3]:
import torch
import torchvision
from torchvision.datasets import CIFAR10
import torchvision.transforms as T
import torch.nn as nn
import os



to_tensor = T.ToTensor()

# means and stds were computed in advanced
ch_means = [0.4914, 0.4822, 0.4465]
ch_stds = [0.2470, 0.2435, 0.2616]

normalize = T.Normalize(mean=ch_means, std=ch_stds)

class tensor(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.t = T.ToTensor()
        
    def forward(self,x):
        return self.t(x)
        

transforms = torch.nn.Sequential(
                                 tensor(),
                                 T.Normalize(mean=ch_means, std=ch_stds)
                                 )



dset = CIFAR10(root = cifar_root,
               transform = transforms,
               download = False)

In [4]:
from torch.utils.data import Dataset

class datasetSplit(Dataset):
    
    def __init__(self, list_IDs, dataset):
        
        self.dataset = dataset
        self.list_IDs = list_IDs
        
        
    def __len__(self):
        
        return len(self.list_IDs)
    
    def __getitem__(self, index):
        
        idx = self.list_IDs[index]
        X, y = self.dataset[idx]
    
        return X, y

        

from torch.utils.data import DataLoader


params = {
          "batch_size": 25,
          "shuffle":  True,
          "num_workers": 2 # increases efficiency
          }



train_len = int(len(dset)*0.8)
idxs_total = torch.randperm(len(dset))

idxs_train = idxs_total[:train_len]
idxs_val = idxs_total[train_len:]

dset_train = datasetSplit(list_IDs=idxs_train, dataset=dset)
dset_val = datasetSplit(list_IDs=idxs_val, dataset=dset)

dloader_train = DataLoader(dataset = dset_train, **params)
dloader_val = DataLoader(dataset = dset_val, **params)

#Model

In [5]:
from torch.nn.functional import relu

class classifier(nn.Module):

    def __init__(self):
        super().__init__()
        
        self.c1 = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, padding="same")
        self.c2 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding="same")
        self.c3 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding="same")

        self.flatten = nn.Flatten(start_dim=1)        
        self.fc1 = nn.Linear(in_features=8*8*256, out_features=1000)
        self.fc2 = nn.Linear(in_features = 1000, out_features=100)
        self.fc3 = nn.Linear(in_features = 100, out_features=10)

        self.m1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.m2 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.drop = nn.Dropout(p=0.3)

    def forward(self, x):
        
        x = self.c1(x)
        x = relu(x)
        x = self.c2(x)
        x = relu(x)
        x = self.m1(x)
        x = self.c3(x)
        x = relu(x)
        x = self.drop(x)
        x = self.m2(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = relu(x)
        x = self.fc2(x)
        x = relu(x)
        x = self.fc3(x)
        
        return x
        

# Training

In [6]:
import torch.optim as optim

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print("working with ", device)

epochs = 15
print_every =  530

p = {"lr": 1.0e-3, "weight_decay": 0}

model = classifier().to(device=device)
cross_entropy = nn.CrossEntropyLoss(reduction="mean")
optimizer = optim.Adam(params=model.parameters(), **p)

model.train(mode=True)
best_val = 0

for e in range(epochs):
    for i, (x_batch, y_batch) in enumerate(dloader_train):
        
        x_batch = x_batch.to(device=device)
        y_batch = y_batch.to(device=device).to(dtype=torch.long)
        
        scores = model(x_batch)
        
        optimizer.zero_grad()
        loss = cross_entropy(scores, y_batch)
        loss.backward()
    
        optimizer.step()

        if (i+1)  % print_every == 0: 
            num_correct = 0
            num_total = 0
            with torch.no_grad():
                
                for j, (x_batch, y_batch) in enumerate(dloader_val):
                    
                    x_batch, y_batch = x_batch.to(device=device), y_batch.to(device=device, dtype=torch.long)
                    
                    scores = model(x_batch)
                    predictions = torch.argmax(scores, dim=-1)
                    num_correct += (predictions == y_batch).sum()
                    num_total += y_batch.shape[0]
                    
                accuracy = num_correct/num_total
                
                if accuracy > best_val: 
                    best_val = accuracy
                    best_model = model
                
                print("epoch %d/%d, validation accuracy %.4f, loss: %.2f " % (e+1, epochs, accuracy, loss.item()) )
                         
        
        

working with  cuda
epoch 1/15, validation accuracy 0.5285, loss: 1.44 
epoch 1/15, validation accuracy 0.6014, loss: 1.13 
epoch 1/15, validation accuracy 0.6626, loss: 0.90 
epoch 2/15, validation accuracy 0.6672, loss: 0.90 
epoch 2/15, validation accuracy 0.6936, loss: 1.10 
epoch 2/15, validation accuracy 0.7137, loss: 0.64 
epoch 3/15, validation accuracy 0.7326, loss: 0.35 
epoch 3/15, validation accuracy 0.7325, loss: 0.35 
epoch 3/15, validation accuracy 0.7344, loss: 0.86 
epoch 4/15, validation accuracy 0.7178, loss: 0.23 
epoch 4/15, validation accuracy 0.7367, loss: 0.55 
epoch 4/15, validation accuracy 0.7406, loss: 0.29 
epoch 5/15, validation accuracy 0.7463, loss: 0.17 
epoch 5/15, validation accuracy 0.7404, loss: 0.24 
epoch 5/15, validation accuracy 0.7479, loss: 0.13 
epoch 6/15, validation accuracy 0.7326, loss: 0.71 
epoch 6/15, validation accuracy 0.7230, loss: 0.13 
epoch 6/15, validation accuracy 0.7382, loss: 0.10 
epoch 7/15, validation accuracy 0.7359, loss:

In [8]:
# Saving model parameters (before colab console closes)

torch.save(best_model.state_dict(), os.path.join(cifar_root,"trained_model.pth"))

In [15]:
dset_test = CIFAR10(root = cifar_root,
                      train=False, 
                     transform=transforms,
                     download=True)

model = classifier()

# loading model parameters 

model.load_state_dict(state_dict=torch.load(os.path.join(cifar_root,"trained_model.pth"), map_location="cpu"))





dloader_test = DataLoader(dataset=dset_test, 
                          batch_size=64)
correct_total = 0

model.to(device=device)
model.eval()

for X_batch, y_batch in dloader_test:

    X_batch, y_batch = X_batch.to(device), y_batch.to(device)
    scores = model(X_batch)
    predicted_labels = torch.argmax(scores, dim=-1)
    correct_total += (predicted_labels == y_batch).sum()
    
accuracy = correct_total/len(dset_test)





Files already downloaded and verified


In [17]:
print(accuracy.item())

0.7328000068664551
